In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date


C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#read in the data
pre=pd.read_csv("before injection mass scan_series4.tsv", sep='\t', index_col=False, skiprows=lambda x: x <=19)

pre["Mass"]=pre['Mass'].round(1)

#rem negative vals
pre[pre < 0] = 0
pre=pre.iloc[::-1]

print(pre.to_string())

    Mass       Current
71   1.0  1.630000e-13
70   1.1  1.579000e-13
69   1.2  2.216000e-13
68   1.3  1.761000e-13
67   1.4  1.391000e-13
66   1.5  1.010000e-13
65   1.6  8.840000e-14
64   1.7  1.486000e-13
63   1.8  1.746000e-13
62   1.9  2.388000e-13
61   2.0  2.820000e-13
60   2.1  1.884000e-13
59   2.2  2.175000e-13
58   2.3  1.598000e-13
57   2.4  1.420000e-13
56   2.5  4.760000e-14
55   2.6  2.370000e-14
54   2.7  1.400000e-15
53   2.8  2.370000e-14
52   2.9  8.700000e-15
51   3.0  2.240000e-14
50   3.1  1.400000e-14
49   3.2  1.730000e-14
48   3.3  4.600000e-15
47   3.4  3.500000e-14
46   3.5  1.120000e-14
45   3.6  3.180000e-14
44   3.7  0.000000e+00
43   3.8  1.970000e-14
42   3.9  1.470000e-14
41   4.0  1.750000e-14
40   4.1  2.420000e-14
39   4.2  9.300000e-15
38   4.3  1.350000e-14
37   4.4  0.000000e+00
36   4.5  1.500000e-14
35   4.6  1.310000e-14
34   4.7  2.020000e-14
33   4.8  2.170000e-14
32   4.9  7.000000e-15
31   5.0  1.420000e-14
30   5.1  1.590000e-14
29   5.2  1

In [3]:
#read in the data
inj=pd.read_csv("during injection RGA scan_series3.tsv", sep='\t', index_col=False, skiprows=lambda x: x <=19)
inj.head()
inj["Mass"]=inj['Mass'].round(1)
##remove negative values
inj[inj < 0] = 0
inj=inj.iloc[::-1]
print(inj.to_string())


    Mass       Current
39   1.0  3.493000e-13
38   1.1  3.870000e-13
37   1.2  3.923000e-13
36   1.3  4.353000e-13
35   1.4  3.220000e-14
34   1.5  0.000000e+00
33   1.6  0.000000e+00
32   1.7  1.397000e-13
31   1.8  1.343000e-13
30   1.9  7.148000e-13
29   2.0  8.062000e-13
28   2.1  7.095000e-13
27   2.2  7.955000e-13
26   2.3  3.977000e-13
25   2.4  2.203000e-13
24   2.5  0.000000e+00
23   2.6  0.000000e+00
22   2.7  0.000000e+00
21   2.8  0.000000e+00
20   2.9  0.000000e+00
19   3.0  0.000000e+00
18   3.1  0.000000e+00
17   3.2  0.000000e+00
16   3.3  0.000000e+00
15   3.4  0.000000e+00
14   3.5  0.000000e+00
13   3.6  0.000000e+00
12   3.7  0.000000e+00
11   3.8  0.000000e+00
10   3.9  0.000000e+00
9    4.0  0.000000e+00
8    4.1  0.000000e+00
7    4.2  0.000000e+00
6    4.3  0.000000e+00
5    4.4  0.000000e+00
4    4.5  0.000000e+00
3    4.6  0.000000e+00
2    4.7  0.000000e+00
1    4.8  0.000000e+00
0    4.9  0.000000e+00


In [4]:
#pressures
#read in the data
p= pd.read_csv("pressures_RGA calibration with N2_series 3", sep='\t', index_col=False)
p.drop(columns=["Live comments","I_emission","I_grid","T-platinum","T-CERNOX","helium"], inplace=True)
p=p.set_axis(['Time', 'injection 100mbar', 'Barion_2', 'Barion_1',"Dual gauge exp","ins vacuum",'injection 1mbar'], axis=1, inplace=False)
#add elapsed time

timestamps=pd.to_datetime(p["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
p.insert(1,"Elapsed time",runtime)
p.head()


,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,Dual gauge exp,ins vacuum,injection 1mbar
0,24/10/2022 11:12:59,0.0,-0.055960,1.990000e-09,6.780000e-10,4.996000e-09,7.614000e-07,0.11287
1,24/10/2022 11:13:07,8.0,-0.055516,2.010000e-09,6.860000e-10,4.996000e-09,7.614000e-07,0.11395
2,24/10/2022 11:13:15,16.0,-0.055787,2.000000e-09,6.870000e-10,4.996000e-09,7.614000e-07,0.11502
3,24/10/2022 11:13:22,23.0,-0.055540,2.010000e-09,6.900000e-10,4.996000e-09,7.614000e-07,0.11609
4,24/10/2022 11:13:30,31.0,-0.059682,1.990000e-09,6.870000e-10,4.996000e-09,7.614000e-07,0.11712


In [5]:
#plotting pressures
plt.figure()
plt.plot((p["Elapsed time"]/3600),p['Barion_2'],marker=".", markersize=5,label='BA2')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Pressure evolution during injection")
plt.yscale('log')

In [6]:
#new dataframe for differential values
data=list(zip(inj["Mass"],inj["Current"]-pre["Current"]))
dif=pd.DataFrame(data, columns=["Mass", "Current"])
dif[dif < 0] = 0
dif=dif.iloc[::-1]
dif.head(10)

,Mass,Current
39,4.9,3.400000e-13
38,4.8,3.735000e-13
37,4.7,3.923000e-13
36,4.6,4.203000e-13
35,4.5,1.910000e-14
34,4.4,0.000000e+00
33,4.3,0.000000e+00
32,4.2,1.327000e-13
31,4.1,1.201000e-13
30,4.0,6.989000e-13


In [7]:
#plot the differential spectrum
plt.figure()
plt.plot((dif["Mass"]),dif['Current'],marker=".", markersize=5)
plt.xlabel('Molecular mass (amu)')
plt.ylabel('Ion current (A)')
plt.legend()
plt.title("Differential spectrum")
plt.yscale('log')
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [18]:
#getting the sensitivity values from the differential spectrum
I_refN2=dif["Current"].max()
print(I_refN2)
p_dif=p["Barion_1"].nlargest(10).mean()-p["Barion_1"].nsmallest(10).mean()
print(p["Barion_2"].iloc[10:20].mean(),p_dif,p["Barion_2"].tail(10).mean())
s_N2=I_refN2/p_dif
print(s_N2)

7.886e-13
1.9750000000000003e-09 4.2388700000000004e-07 9.123699999999999e-07
1.8604014749213822e-06
